In [1]:
import pandas as pd
dataset = pd.read_csv(r'C:\Users\divye\Desktop\symptom-diagnosis\data\dataset.csv')
symptom_severity = pd.read_csv(r'C:\Users\divye\Desktop\symptom-diagnosis\data\Symptom-severity.csv')
precaution_data = pd.read_csv(r'C:\Users\divye\Desktop\symptom-diagnosis\data\symptom_precaution.csv')
description_data = pd.read_csv(r'C:\Users\divye\Desktop\symptom-diagnosis\data\symptom_Description.csv')

In [2]:
def data_exploration(df,name):
    print(f"\n{name} Information")
    print(f"shape{df.shape}")
    print(f"DataTypes:\n")
    print(df.dtypes)
    print(f"Missing values:\n")
    print(df.isnull().sum())
    print("\nDescriptive Statistics:")
    display(df.describe(include='all'))

data_exploration(dataset,"maindataset")
data_exploration(description_data,"Descriptiondataset")
data_exploration(precaution_data,"precautiondataset")
data_exploration(symptom_severity,"severitydataste")    



maindataset Information
shape(4920, 18)
DataTypes:

Disease       object
Symptom_1     object
Symptom_2     object
Symptom_3     object
Symptom_4     object
Symptom_5     object
Symptom_6     object
Symptom_7     object
Symptom_8     object
Symptom_9     object
Symptom_10    object
Symptom_11    object
Symptom_12    object
Symptom_13    object
Symptom_14    object
Symptom_15    object
Symptom_16    object
Symptom_17    object
dtype: object
Missing values:

Disease          0
Symptom_1        0
Symptom_2        0
Symptom_3        0
Symptom_4      348
Symptom_5     1206
Symptom_6     1986
Symptom_7     2652
Symptom_8     2976
Symptom_9     3228
Symptom_10    3408
Symptom_11    3726
Symptom_12    4176
Symptom_13    4416
Symptom_14    4614
Symptom_15    4680
Symptom_16    4728
Symptom_17    4848
dtype: int64

Descriptive Statistics:


,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
count,4920,4920,4920,4920,4572,3714,2934,2268,1944,1692,1512,1194,744,504,306,240,192,72
unique,41,34,48,54,50,38,32,26,21,22,21,18,11,8,4,3,3,1
top,Fungal infection,vomiting,vomiting,fatigue,high_fever,headache,nausea,abdominal_pain,abdominal_pain,yellowing_of_eyes,yellowing_of_eyes,irritability,malaise,muscle_pain,chest_pain,chest_pain,blood_in_sputum,muscle_pain
freq,120,822,870,726,378,348,390,264,276,228,198,120,126,72,96,144,72,72



Descriptiondataset Information
shape(41, 2)
DataTypes:

Disease        object
Description    object
dtype: object
Missing values:

Disease        0
Description    0
dtype: int64

Descriptive Statistics:


,Disease,Description
count,41,41
unique,41,41
top,Drug Reaction,An adverse drug reaction (ADR) is an injury ca...
freq,1,1



precautiondataset Information
shape(41, 5)
DataTypes:

Disease         object
Precaution_1    object
Precaution_2    object
Precaution_3    object
Precaution_4    object
dtype: object
Missing values:

Disease         0
Precaution_1    0
Precaution_2    0
Precaution_3    1
Precaution_4    1
dtype: int64

Descriptive Statistics:


,Disease,Precaution_1,Precaution_2,Precaution_3,Precaution_4
count,41,41,41,40,40
unique,41,32,34,30,24
top,Drug Reaction,Consult nearest hospital,exercise,consult doctor,follow up
freq,1,3,3,6,6



severitydataste Information
shape(133, 2)
DataTypes:

Symptom    object
weight      int64
dtype: object
Missing values:

Symptom    0
weight     0
dtype: int64

Descriptive Statistics:


,Symptom,weight
count,133,133.000000
unique,132,NaN
top,fluid_overload,NaN
freq,2,NaN
mean,NaN,4.225564
std,NaN,1.323543
min,NaN,1.000000
25%,NaN,3.000000
50%,NaN,4.000000
75%,NaN,5.000000


In [3]:
# Clean symptom names (remove extra spaces, standardize formatting)
def clean_symptom_names(df):
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].str.strip().str.lower()
    return df

dataset = clean_symptom_names(dataset)
symptom_severity = clean_symptom_names(symptom_severity)

# Check for duplicates
print("Duplicate rows in main dataset:", dataset.duplicated().sum())

# Remove duplicates if any
dataset.drop_duplicates(inplace=True)

Duplicate rows in main dataset: 4616


In [4]:

# Merge symptom severity with main dataset
# Assuming symptoms are in columns Symptom1, Symptom2, etc. in the main dataset
symptom_cols = [col for col in dataset.columns if 'Symptom' in col]

for col in symptom_cols:
    dataset = dataset.merge(symptom_severity, how='left', 
                           left_on=col, right_on='Symptom', 
                           suffixes=('', f'_{col}'))
    dataset.drop(columns=['Symptom'], inplace=True)  # Drop duplicate column

# Calculate total severity score
severity_cols = [col for col in dataset.columns if 'weight' in col]
dataset['Total_Severity'] = dataset[severity_cols].sum(axis=1)

In [5]:
from sklearn.preprocessing import LabelEncoder
# Encode disease names (target variable)
label_encoder = LabelEncoder()
dataset['Disease_Encoded'] = label_encoder.fit_transform(dataset['Disease'])

In [6]:

from sklearn.model_selection import train_test_split
# Assuming 'Disease_Encoded' is our target variable
X = dataset.drop(columns=['Disease', 'Disease_Encoded'])
y = dataset['Disease_Encoded']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set size: {X_train.shape[0]}")
print(f"Testing set size: {X_test.shape[0]}")

Training set size: 248
Testing set size: 63


In [7]:

# Save cleaned and processed datasets
dataset.to_csv('cleaned_dataset.csv', index=False)
X_train.to_csv('X_train.csv', index=False)
X_test.to_csv('X_test.csv', index=False)
y_train.to_csv('y_train.csv', index=False)
y_test.to_csv('y_test.csv', index=False)

In [8]:
df=pd.read_csv(r"C:\Users\divye\Desktop\symptom-diagnosis\cleaned_dataset.csv")

In [9]:
# Handle missing values: most frequent for categorical, median for numerical
for col in df.columns:
    if df[col].dtype == 'object':
        mode_val = df[col].mode()
        if not mode_val.empty:
            df[col] = df[col].fillna(mode_val[0])
        else:
            df[col] = df[col].fillna("Unknown")  # Or any default string
    else:
        median_val = df[col].median()
        if pd.notna(median_val):
            df[col] = df[col].fillna(median_val)
        else:
            df[col] = df[col].fillna(0)  # Default numeric fallback if all values are NaN

# Convert symptoms to numerical values (except 'Disease')
for col in df.columns:
    if df[col].dtype == 'object' and col != 'Disease':
        df[col] = df[col].astype('category').cat.codes


In [10]:
# Encode target variable
le = LabelEncoder()
df['Disease'] = le.fit_transform(df['Disease'])

In [11]:
print("\nMissing values after treatment:")
print(df.isnull().sum())


Missing values after treatment:
Disease              0
Symptom_1            0
Symptom_2            0
Symptom_3            0
Symptom_4            0
Symptom_5            0
Symptom_6            0
Symptom_7            0
Symptom_8            0
Symptom_9            0
Symptom_10           0
Symptom_11           0
Symptom_12           0
Symptom_13           0
Symptom_14           0
Symptom_15           0
Symptom_16           0
Symptom_17           0
weight               0
weight_Symptom_2     0
weight_Symptom_3     0
weight_Symptom_4     0
weight_Symptom_5     0
weight_Symptom_6     0
weight_Symptom_7     0
weight_Symptom_8     0
weight_Symptom_9     0
weight_Symptom_10    0
weight_Symptom_11    0
weight_Symptom_12    0
weight_Symptom_13    0
weight_Symptom_14    0
weight_Symptom_15    0
weight_Symptom_16    0
weight_Symptom_17    0
Total_Severity       0
Disease_Encoded      0
dtype: int64


In [12]:
from sklearn.preprocessing import StandardScaler
# Split data
X = df.drop('Disease', axis=1)
y = df['Disease']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22,stratify=y)

# Scale data (for models that need it)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [13]:
y

0      15
1      15
2      15
3      15
4      15
       ..
306    28
307    28
308    28
309    28
310    28
Name: Disease, Length: 311, dtype: int64

In [14]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((248, 36), (63, 36), (248,), (63,))

In [15]:
# X_train_scaled

In [16]:
y_train.value_counts()

Disease
3     12
11     8
22     8
31     8
8      8
29     7
26     7
23     7
37     7
12     7
38     7
10     7
20     7
35     7
25     7
19     7
36     6
6      6
34     6
21     6
32     6
40     6
27     6
17     6
30     6
9      6
13     5
24     5
28     5
0      5
5      5
14     5
7      5
4      4
16     4
33     4
1      4
18     4
2      4
15     4
39     4
Name: count, dtype: int64

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Train
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_scaled, y_train)

# Evaluate
y_pred = lr.predict(X_test_scaled)
print("\nLogistic Regression Results:")
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Logistic Regression Results:
Accuracy: 0.8412698412698413
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         1
           3       1.00      1.00      1.00         3
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         1
           6       1.00      1.00      1.00         1
           7       0.50      1.00      0.67         1
           8       1.00      1.00      1.00         2
           9       0.33      0.50      0.40         2
          10       1.00      1.00      1.00         2
          11       1.00      1.00      1.00         2
          12       1.00      0.50      0.67         2
          13       1.00      1.00      1.00         1
          14       1.00      1.00      1.00         1
          15       1.00      1.00      1.00         1
          16       0.5

c:\instal_file_location\envs\symptom-diagnosis\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\instal_file_location\envs\symptom-diagnosis\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\instal_file_location\envs\symptom-diagnosis\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita

In [18]:

from sklearn.ensemble import RandomForestClassifier
import numpy as np

# Train (no scaling needed)
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)

# Evaluate
y_pred = rf.predict(X_test)
print("\nRandom Forest Results:")
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Feature importance
importances = rf.feature_importances_
print("\nTop 5 Important Features:")
for i in np.argsort(importances)[-5:]:
    print(f"{X.columns[i]}: {importances[i]:.4f}")


Random Forest Results:
Accuracy: 0.9841269841269841
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         1
           3       1.00      1.00      1.00         3
           4       1.00      1.00      1.00         1
           5       1.00      1.00      1.00         1
           6       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         1
           8       1.00      1.00      1.00         2
           9       1.00      1.00      1.00         2
          10       1.00      1.00      1.00         2
          11       1.00      1.00      1.00         2
          12       1.00      1.00      1.00         2
          13       1.00      1.00      1.00         1
          14       1.00      1.00      1.00         1
          15       1.00      1.00      1.00         1
          16       0.50     

c:\instal_file_location\envs\symptom-diagnosis\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\instal_file_location\envs\symptom-diagnosis\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\instal_file_location\envs\symptom-diagnosis\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita

In [22]:
import joblib
from sklearn.preprocessing import LabelEncoder

# Encode disease names
label_encoder = LabelEncoder()
dataset['Disease_Encoded'] = label_encoder.fit_transform(dataset['Disease'])

# Save all artifacts together
joblib.dump((rf, scaler, X_train.columns, label_encoder), 'symptom_diagnosis_model.pkl')

# Verify saving
print("Model, scaler, feature names, and label encoder saved successfully!")


Model, scaler, feature names, and label encoder saved successfully!


In [28]:
import pandas as pd
import joblib

# Load artifacts
model, scaler, feature_names, label_encoder = joblib.load('symptom_diagnosis_model.pkl')

# User input (replace with actual input values)
new_symptoms = pd.DataFrame([{
    'Symptom1': 3,
    'Symptom2': 7,
    'Symptom3': 12,
    'Symptom4': 5,
    'Symptom5': 9,
    'Total_Severity': 36  # Sum of severity weights
}], columns=feature_names)

# Scale the input (if scaling was applied during training)
new_symptoms_scaled = scaler.transform(new_symptoms)

# Predict the disease
predicted_label = model.predict(new_symptoms_scaled)
predicted_disease = label_encoder.inverse_transform(predicted_label)

print("Predicted Disease:", predicted_disease[0])


Predicted Disease: fungal infection


c:\instal_file_location\envs\symptom-diagnosis\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [27]:
# Example for testing
test_input = pd.DataFrame([{
    'Symptom1': 5,  # Example encoded values for symptoms
    'Symptom2': 8,
    'Symptom3': 3,
    'Symptom4': 2,
    'Symptom5': 1,
    'Total_Severity': 19
}], columns=feature_names)

test_scaled = scaler.transform(test_input)
prediction = model.predict(test_scaled)
print("Predicted Disease:", label_encoder.inverse_transform(prediction)[0])


Predicted Disease: fungal infection


c:\instal_file_location\envs\symptom-diagnosis\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
